# Generate daily prices using daily data service - C#

### Overview
Daily prices is designed to illustrate price movements for an instrument over a look back period.

This sample demonstrates how to directly request and plot daily prices.<br>

It enables to retrieve **on-demand** daily prices data by calling a dedicated service on a **daily** data store. 

### Inputs/outputs
Daily prices sample requires instrument's identifier as per input. It returns daily prices and volumes.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily prices sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Daily.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress($"https://{System.Environment.GetEnvironmentVariable("GRPC_APIS")}");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [ ]:
// Set instrument
var ticker = "AAPL";
var exchange = "XNGS";

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily prices service
var service = new DailyPricesService.DailyPricesServiceClient(channel);

// Create and process prices data request
var request = new DailyPricesRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}
};

var reply = service.DailyPrices(request, headers);

In [ ]:
//Visualize request results
display(reply.Data)

### Step 4: Visualize data

#### 4.1 Plot the prices and volumes
The following code snippets retrive *daily prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Extract the daily prices and the volumes
var prices = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Price));
var volumes = reply.Data.Select(t => new Tuple<DateTime, long>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume));

In [ ]:
// Create the graph
var pricesAndVolumes = new List<Scattergl>
{
    new Scattergl
    {
        name = "Price",
        mode = "lines",
        x = prices.Select(c => c.Item1),
        y = prices.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Volume",
        mode = "bar",
        x = volumes.Select(v => v.Item1),
        y = volumes.Select(v => v.Item2),
        yaxis = "y2"
    } 
};


var layout = new Layout.Layout{ 
    title = $"Daily price & Volume for {ticker}",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Volume", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(pricesAndVolumes, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);